# Make sure runtime is set to GPU

# Install necessary packages

In [1]:
!pip install tensorflow-gpu
!pip install PyDrive

You are using pip version 9.0.1, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

class download_data_from_folder(object):
    def __init__(self,path):
        path_id = path[path.find('id=')+3:]
        self.file_list = self.get_files_in_location(path_id)
        self.unwrap_data(self.file_list)
    def get_files_in_location(self,folder_id):
        file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()
        return file_list
    def unwrap_data(self,file_list,directory='.'):
        for i, file in enumerate(file_list):
            print(str((i + 1) / len(file_list) * 100) + '% done copying')
            if file['mimeType'].find('folder') != -1:
                if not os.path.exists(os.path.join(directory, file['title'])):
                    os.makedirs(os.path.join(directory, file['title']))
                print('Copying folder ' + os.path.join(directory, file['title']))
                self.unwrap_data(self.get_files_in_location(file['id']), os.path.join(directory, file['title']))
            else:
                if not os.path.exists(os.path.join(directory, file['title'])):
                    downloaded = drive.CreateFile({'id': file['id']})
                    downloaded.GetContentFile(os.path.join(directory, file['title']))
        return None

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
data_path = 'https://drive.google.com/open?id=1oFxTLOPF46IQljQYH0UFPg59nIvwInrY'
download_data_from_folder(data_path)

## Load modules

In [1]:
import sys
import os, glob
sys.path.append(os.path.join('..','ext','neuron'))
import numpy as np
from Zip_data import Unzip_class
from Utils import normalize, visualize_model, create_model, train, load_atlas, data_generator

Using TensorFlow backend.
c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [ ]:
%load_ext tensorboard.notebook

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
data_path = os.path.abspath('//content//gdrive//My Drive//AAPM Summer School//code')
os.chdir(data_path)
print(os.path.abspath(os.curdir))

## Define model location and registered data
Also unzip data if we haven't already

In [2]:
data_dir = os.path.join('..','Reg_Data')
if not os.path.exists(data_dir):
    print('Unzipping data')
    Unzip_class(os.path.join('..','Reg_Data'),r'..')
    print('Finished unzipping')
model_dir = os.path.join('..','models')

## Load in the atlas file

In [4]:
atlas_file= os.path.join('..','reg_data','Atlas_Data.npy')
atlas_vol = load_atlas(atlas_file)

### Creating your model

This is the part that you have the most unique control over, what exactly will your model look like?

One good way of visualizing the model is to create it and look at the graphical architecture with tensorboard

Lets start off with something simple, a UNet with one concatentation layer

In [5]:
layers = {'Layer_0':{'Encoding':[16,32],'Decoding':[32,16,8]},
          'Base':{'Encoding':[64]}}
model_desc = 'Shallow_net' # Name of your model
# The numbers inside are the number of filter banks, you can have mulitple filter banks per layer

This should be imagined as a U net, the numbers within [] are the number of filters in the convolution blocks, the network will first perform convolutions downthe encoding side to the base, and then concatenate from the previous encoding layer before performing decoding convolutions

This might not look like much, but lets see what it looks like graphically

## Visualize the model

In [6]:
visualize_model(layers, atlas_vol.shape[1:-1], model_desc)

Model created at: K:\Morfeus\AAPM_SummerSchool\bma_voxel_morph\Tensorboard_models\Shallow_net


In [ ]:
logs_base_dir = os.path.join('..','Tensorboard_models')
%tensorboard --logdir {logs_base_dir}

## Load training data

In [7]:
train_generator = data_generator(atlas_vol,data_dir)
print('We have ' + str(len(train_generator)) + ' registrations available')

We have 116 registrations available


## Define hyperparameters
These are variables which you will manipulate in order to create the best model possible

In [8]:
learning_rate = 0.001 # Rate at which our gradients will change during each back propogation, typically in range of 1e-2 to 1e-5
number_of_epochs = 10 # The number of epochs to be trained, one epoch means that you have seen the entirety of your dataset
                      # However, since we defined steps per epoch this might not apply
regularization_parameter = 0.01 # Lambda in regularization equation
steps_per_epoch = 30
loss_function = 'mse'
batch_normalization = True

## Create the model

In [9]:
model, callbacks = create_model(layers, atlas_vol.shape[1:-1], model_desc, batch_norm=batch_normalization, data_generator=train_generator)

In [ ]:
train(model, train_generator, callbacks, learning_rate, number_of_epochs, 
      regularization_parameter, steps_per_epoch,loss_function)

Epoch 1/10
14/30 [=============>................] - ETA: 10:15 - loss: 0.0334 - spatial_transformer_1_loss: 0.0334 - flow_loss: 1.1482e-0 - ETA: 5:21 - loss: 0.0302 - spatial_transformer_1_loss: 0.0302 - flow_loss: 8.1864e-0 - ETA: 3:41 - loss: 0.0268 - spatial_transformer_1_loss: 0.0268 - flow_loss: 2.6281e- - ETA: 2:51 - loss: 0.0249 - spatial_transformer_1_loss: 0.0249 - flow_loss: 5.6768e- - ETA: 2:20 - loss: 0.0236 - spatial_transformer_1_loss: 0.0236 - flow_loss: 0.0010   - ETA: 1:59 - loss: 0.0229 - spatial_transformer_1_loss: 0.0229 - flow_loss: 0.00 - ETA: 1:43 - loss: 0.0215 - spatial_transformer_1_loss: 0.0215 - flow_loss: 0.00 - ETA: 1:31 - loss: 0.0211 - spatial_transformer_1_loss: 0.0210 - flow_loss: 0.00 - ETA: 1:21 - loss: 0.0209 - spatial_transformer_1_loss: 0.0208 - flow_loss: 0.00 - ETA: 1:13 - loss: 0.0223 - spatial_transformer_1_loss: 0.0222 - flow_loss: 0.00 - ETA: 1:06 - loss: 0.0222 - spatial_transformer_1_loss: 0.0221 - flow_loss: 0.00 - ETA: 1:00 - loss: 0.021

## Monitor progress and visualize

In [ ]:
logs_base_dir = os.path.join('..','Tensorboard_models')
%tensorboard --logdir {logs_base_dir}

Play around! Add more layers, convolutions, see if you can reduce your loss the most, change the bottom two files, make sure to rename your model

In [14]:
layers = {'Layer_0':{'Encoding':[16],'Decoding':[16,16]},
          'Layer_1':{'Encoding':[32],'Decoding':[32,32]},
          'Layer_2':{'Encoding':[32],'Decoding':[32]},
          'Layer_3':{'Encoding':[32],'Decoding':[32]},
          'Base':{'Encoding':[32]}}
model_desc = 'Deep_net' # Name of your model
# The numbers inside are the number of filter banks, you can have mulitple filter banks per layer
visualize_model(layers, atlas_vol.shape[1:-1], model_desc)

View your model by scrolling up to look at the tensorboard module

In [15]:
model, callbacks = create_model(layers, atlas_vol.shape[1:-1], model_desc, batch_norm=batch_normalization, data_generator=train_generator)

K:\Morfeus\AAPM_SummerSchool\bma_voxel_morph\Tensorboard_models\Deep_net


In [ ]:
train(model, train_generator, callbacks, learning_rate, number_of_epochs, 
      regularization_parameter, steps_per_epoch,loss_function)